In [5]:
import requests
import datetime
import json
from datetime import timedelta
import pandas as pd
from IPython.display import display
from dateutil.relativedelta import relativedelta
import seaborn as sns
pd.options.mode.chained_assignment = None 
from tqdm import tqdm
from tabulate import tabulate

In [6]:
def create_options(query, skip_iterator):
    return query.replace('skip_param', str(skip_iterator))

def get_data_from_sg(query, url, name, amonths=6):
    result = []
    headers = {
        'authority': 'api.thegraph.com',
        'content-type': 'application/json',
        'accept': '*/*',
        'origin': 'https://thegraph.com',
        'sec-fetch-site': 'same-site',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://thegraph.com/',
    }
    timestamp = int(datetime.datetime.now().timestamp())
    timestampend = int((datetime.datetime.now()- relativedelta(months=amonths)).timestamp())
    while(timestamp >= timestampend):
        data = create_options(query, timestamp)
        response = requests.post(f'https://api.thegraph.com/subgraphs/name/{url}', headers = headers, data=data)
        response = response.json()["data"][name]
        if response:
            result.append(response)     
            timestamp = int(response[-1]['timestamp'])
        else:
            break
    return [item for sublist in result for item in sublist]    

In [12]:
# GMX get date from AUM (asset under management) and supply GLP for price calculation
query = '{"query":"{\\n glpStats(first: 1000, orderBy: timestamp, orderDirection: desc, where : {timestamp_lt : skip_param}) {\\n period\\n aumInUsdg\\n glpSupply\\n timestamp\\n}\\n}\\n","variables":null}'
url = 'gmx-io/gmx-avalanche-stats'
name = 'glpStats'
price_buff = get_data_from_sg(query, url, name)
price = pd.DataFrame(price_buff)

price['date'] = pd.to_datetime(price['timestamp'], unit = 's')

In [13]:
# price calculation

price["price"] = (price['aumInUsdg'].apply(int)) / (price['glpSupply'].apply(int))

In [14]:
price.to_csv("price_0709.csv")

In [7]:
# GMX get date from poolAmount
query = '{"query":"{\\n tokenStats(first: 1000, orderBy: timestamp, orderDirection: desc, where : {timestamp_lt : skip_param}) {\\n period\\n poolAmount\\n poolAmountUsd\\n timestamp\\n token\\n}\\n}\\n","variables":null}'
url = 'gmx-io/gmx-avalanche-stats'
name = 'tokenStats'
tokens_buff = get_data_from_sg(query, url, name)
tokens = pd.DataFrame(tokens_buff)
tokens['date'] = pd.to_datetime(tokens['timestamp'], unit = 's')

In [9]:
tokens.to_csv("tokens_0709.csv")

In [10]:
GLP = {
  '0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7': 'AVAX',
  '0x49d5c2bdffac6ce2bfdb6640f4f80f226bc10bab': 'WETH.e',
  '0x50b7545627a5162f82a992c33b87adc75187b218': 'WBTC.e',
  '0x152b9d0fdc40c096757f570a51e494bd4b943e50': 'BTC.b',
  '0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664': 'USDC.e',
  '0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e': 'USDC',
  '0x130966628846bfd36ff31a822705796e8cb8c18d': 'MIM',}
      

In [15]:
price = price[price.period == 'daily']
tokens = tokens[tokens.period == 'daily']

In [16]:
# calculation of weights for each token
stats_weight = []
    
for _, row in price.iterrows():
    
    date = row['timestamp']
    aum = row['aumInUsdg']
    tvl = tokens[tokens["timestamp"] == date]["poolAmountUsd"].apply(int).sum()
    for _, rowt in tokens[tokens["timestamp"] == date].iterrows():
        rowt["weight"] = 100 * int(rowt['poolAmountUsd']) / tvl
        rowt["token_name"] = GLP[rowt["token"]]
        stats_weight.append({
            'date' : rowt['date'],
            'token' : rowt['token'],
            'token_name' : rowt['token_name'],
            'weight' : rowt['weight'],
        })
stats_weight = pd.DataFrame(stats_weight)
stats_weight

,date,token,token_name,weight
0,2022-09-07,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e,USDC,33.974179
1,2022-09-07,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,AVAX,8.019466
2,2022-09-07,0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664,USDC.e,29.179299
3,2022-09-07,0x50b7545627a5162f82a992c33b87adc75187b218,WBTC.e,7.048561
4,2022-09-07,0x49d5c2bdffac6ce2bfdb6640f4f80f226bc10bab,WETH.e,16.266363
...,...,...,...,...
1047,2022-02-28,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,AVAX,12.620951
1048,2022-02-28,0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664,USDC.e,50.175494
1049,2022-02-28,0x50b7545627a5162f82a992c33b87adc75187b218,WBTC.e,18.107748
1050,2022-02-28,0x49d5c2bdffac6ce2bfdb6640f4f80f226bc10bab,WETH.e,18.317507


In [17]:
stats_weight.to_csv("stats_weight_0709.csv")

In [20]:
AAVE = {
    '0x50b7545627a5162f82a992c33b87adc75187b218': {'name': 'WBTC.e', 'decimals': 8, "id": 0},
    '0x49d5c2bdffac6ce2bfdb6640f4f80f226bc10bab': {'name': 'WETH.e', 'decimals': 18}, 
    '0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7': {'name': 'WAVAX', 'decimals': 18}}

In [21]:
# AAVE get date from borrow rate

idr =  "0x49d5c2bdffac6ce2bfdb6640f4f80f226bc10bab0xa97684ead0e402dc232d5a977953df7ecbab3cdb"
query = '{"query":"{\\n reserveParamsHistoryItems(first: 1000, orderBy: timestamp, orderDirection: desc, where : {reserve: \\"'+idr+'\\", timestamp_lt : skip_param}) {\\n variableBorrowRate\\n stableBorrowRate\\n priceInEth\\n priceInUsd\\n timestamp\\n}\\n}\\n"}'
#query = '{query: "{\\n  \\n  reserveParamsHistoryItems \\n  {\\n    variableBorrowRate\\n    stableBorrowRate\\n    priceInEth\\n    priceInUsd\\n    timestamp\\n    \\n  }\\n  \\n  }\\n  \\n  \\n","variables":null}'
url = 'aave/protocol-v3-avalanche'
name = 'reserveParamsHistoryItems'
eth_buff = get_data_from_sg(query, url, name)
eth_rate = pd.DataFrame(eth_buff)

eth_rate['date'] = pd.to_datetime(eth_rate['timestamp'], unit = 's')

In [27]:
eth_rate['rate']= (eth_rate['variableBorrowRate'].apply(int)) / 10 ** 27

In [28]:
eth_rate

,variableBorrowRate,stableBorrowRate,priceInEth,priceInUsd,timestamp,date,rate
0,53393486807981862374561020,90000000000000000000000000,157639127724,157639127724,1662578017,2022-09-07 19:13:37,0.053393
1,53257810197867614908344384,90000000000000000000000000,157353000000,157353000000,1662577041,2022-09-07 18:57:21,0.053258
2,53259992288235175372144500,90000000000000000000000000,157353000000,157353000000,1662577041,2022-09-07 18:57:21,0.05326
3,53257810197867614908344384,90000000000000000000000000,156513175322,156513175322,1662575170,2022-09-07 18:26:10,0.053258
4,53258448372430894571751071,90000000000000000000000000,156513175322,156513175322,1662574798,2022-09-07 18:19:58,0.053258
...,...,...,...,...,...,...,...
30519,0,90000000000000000000000000,275430239973,275430239973,1647500046,2022-03-17 06:54:06,0.0
30520,0,90000000000000000000000000,277165000000,277165000000,1647483348,2022-03-17 02:15:48,0.0
30521,0,90000000000000000000000000,274890700407,274890700407,1647471353,2022-03-16 22:55:53,0.0
30522,0,90000000000000000000000000,275809681711,275809681711,1647461648,2022-03-16 20:14:08,0.0


In [43]:
eth_rate['rate'] = eth_rate.rate.apply(float)

In [45]:
eth_rate['rate'].describe()

count    30524.000000
mean         0.012064
std          0.007378
min          0.000000
25%          0.008601
50%          0.010318
75%          0.012580
max          0.060576
Name: rate, dtype: float64

In [46]:
eth_rate['rate'].median()

0.01031808488191893

In [31]:
eth_rate.to_csv("eth_rate_0709.csv")

In [32]:
idr =  "0xb31f66aa3c1e785363f0875a1b74e27b85fd66c70xa97684ead0e402dc232d5a977953df7ecbab3cdb"
query = '{"query":"{\\n reserveParamsHistoryItems(first: 1000, orderBy: timestamp, orderDirection: desc, where : {reserve: \\"'+idr+'\\", timestamp_lt : skip_param}) {\\n variableBorrowRate\\n stableBorrowRate\\n priceInEth\\n priceInUsd\\n timestamp\\n}\\n}\\n"}'
#query = '{query: "{\\n  \\n  reserveParamsHistoryItems \\n  {\\n    variableBorrowRate\\n    stableBorrowRate\\n    priceInEth\\n    priceInUsd\\n    timestamp\\n    \\n  }\\n  \\n  }\\n  \\n  \\n","variables":null}'
url = 'aave/protocol-v3-avalanche'
name = 'reserveParamsHistoryItems'
avax_buff = get_data_from_sg(query, url, name)
avax_rate = pd.DataFrame(avax_buff)

avax_rate['date'] = pd.to_datetime(avax_rate['timestamp'], unit = 's')

In [34]:
avax_rate['rate']= (avax_rate['variableBorrowRate'].apply(int)) / 10 ** 27

In [52]:
avax_rate['rate'] = avax_rate.rate.apply(float)

In [53]:
avax_rate['rate'].describe()

count    125302.000000
mean          0.062592
std           0.046123
min           0.000000
25%           0.055766
50%           0.060605
75%           0.066201
max           2.907122
Name: rate, dtype: float64

In [54]:
avax_rate.rate.median()

0.06060528608806464

In [36]:
avax_rate.to_csv("avax_rate_0709.csv")

In [37]:
idr =  "0x50b7545627a5162f82a992c33b87adc75187b2180xa97684ead0e402dc232d5a977953df7ecbab3cdb"
query = '{"query":"{\\n reserveParamsHistoryItems(first: 1000, orderBy: timestamp, orderDirection: desc, where : {reserve: \\"'+idr+'\\", timestamp_lt : skip_param}) {\\n variableBorrowRate\\n stableBorrowRate\\n priceInEth\\n priceInUsd\\n timestamp\\n}\\n}\\n"}'
#query = '{query: "{\\n  \\n  reserveParamsHistoryItems \\n  {\\n    variableBorrowRate\\n    stableBorrowRate\\n    priceInEth\\n    priceInUsd\\n    timestamp\\n    \\n  }\\n  \\n  }\\n  \\n  \\n","variables":null}'
url = 'aave/protocol-v3-avalanche'
name = 'reserveParamsHistoryItems'
btc_buff = get_data_from_sg(query, url, name)
btc_rate = pd.DataFrame(btc_buff)

btc_rate['date'] = pd.to_datetime(btc_rate['timestamp'], unit = 's')

In [39]:
btc_rate['rate']= (btc_rate['variableBorrowRate'].apply(int)) / 10 ** 27

In [47]:
btc_rate['rate'] = btc_rate.rate.apply(float)

In [49]:
btc_rate['rate'].median()

0.004561721259657841

In [50]:
btc_rate

,variableBorrowRate,stableBorrowRate,priceInEth,priceInUsd,timestamp,date,rate
0,17346442436278160106870522,90000000000000000000000000,1909092000000,1909092000000,1662577166,2022-09-07 18:59:26,0.017346
1,17346497520593609417527182,90000000000000000000000000,1897586000000,1897586000000,1662575949,2022-09-07 18:39:09,0.017346
2,17356832384088522129399856,90000000000000000000000000,1897586000000,1897586000000,1662575687,2022-09-07 18:34:47,0.017357
3,17356797752785251771244404,90000000000000000000000000,1892748230284,1892748230284,1662571815,2022-09-07 17:30:15,0.017357
4,17357289321007201662647447,90000000000000000000000000,1892748230284,1892748230284,1662571759,2022-09-07 17:29:19,0.017357
...,...,...,...,...,...,...,...
17562,0,90000000000000000000000000,4117849133087,4117849133087,1647483259,2022-03-17 02:14:19,0.000000
17563,0,90000000000000000000000000,4110903985102,4110903985102,1647482206,2022-03-17 01:56:46,0.000000
17564,0,90000000000000000000000000,4102297330567,4102297330567,1647481788,2022-03-17 01:49:48,0.000000
17565,0,90000000000000000000000000,4097345000000,4097345000000,1647471313,2022-03-16 22:55:13,0.000000


In [41]:
btc_rate.to_csv("btc_rate_0709.csv")